In [25]:
import re
import requests
from Bio import SeqIO
from functools import lru_cache
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils'

def read_fasta(file, raw=False) -> dict:
    """
    Reads information from a fasta formatted file.
    """
    seq_raw = SeqIO.parse(file, "fasta")
    if raw : return seq_raw
    seq_dict = {}
    for seq in seq_raw:
        raw_id = seq.id.split("|")
        if len(raw_id) > 2:
            id = raw_id[3]
            seq.id = raw_id[3]
        elif len(raw_id) == 2:
            if raw_id[0] == raw_id[1]: id = raw_id[0]
            else: id = seq.id
        else:
            id = seq.id
        seq_dict[id] = seq
    return seq_dict

def get_ncbi(id: str):
    params = {
        'db': 'nucleotide',
        'id': id,
        'retmode': 'text',
        'rettype': 'text'
    }
    r = requests.get(f'{url}/efetch.fcgi', params=params)
    try:
        match = re.search(r'strain\s(.[^\s]+)|strain="(.[^\s]+)"|subtype:\s(.[^\s;"]*)|(subtype\s[A-Z])', r.text)
        result = "".join(match.groups(""))
        if 'subtype' not in result:
            if 'CRF' not in result: 
                result = 'subtype ' + result
        return result
    except AttributeError:
        if 'NL4-3' in r.text: return 'NL4-3'
        elif 'HXB2' in r.text: return 'HXB2'
        else: return None

def serch_hivdb(seq):
    payload = {
        'operationName': 'guess_type',
        'query': 'query example($sequences: [UnalignedSequenceInput]!) {currentVersion { text, publishDate },sequenceAnalysis(sequences: $sequences) {bestMatchingSubtype { displayWithoutDistance, distance },}}',
        'variables': {
            'sequences': [{'header': id, 'sequence': str(seq)}]
        }
    }
    hivdb = requests.post('https://hivdb.stanford.edu/graphql', json=payload)
    best_match = hivdb.json().get('data').get('sequenceAnalysis')[0].get('bestMatchingSubtype')
    subtype = 'subtype ' + best_match.get('displayWithoutDistance')
    conf = (1 - float(best_match.get('distance')))*100
    return f'{subtype} ({conf}%)'
     
@lru_cache(maxsize=128)
def get_subtype(id: str, fasta=None):
    result = get_ncbi(id)
    if result: return result
    if fasta: return serch_hivdb(fasta)
    return 'UNKNOWN'


In [ ]:
id = 'X01762.1'
params = {
    'db': 'nucleotide',
    'id': id,
    'retmode': 'text',
    'rettype': 'text'
}
r = requests.get(f'{url}/efetch.fcgi', params=params)
try:
    match = re.search(r'strain\s(.[^\s]+)|strain="(.[^\s]+)"|subtype:\s(.[^\s;"]*)|(subtype\s[A-Z])', r.text)
    print(''.join(match.groups('')))
except AttributeError:
    if 'NL4-3' in r.text: 
        print(f'{id} - NL4-3')
    else:
        payload = {
            'operationName': 'guess_type',
            'query': 'query example($sequences: [UnalignedSequenceInput]!) {currentVersion { text, publishDate },sequenceAnalysis(sequences: $sequences) {bestMatchingSubtype { display },}}',
            'variables': {
                'sequences': [{'header': id, 'sequence': str(fa[id].seq)}]
            }
        }
        hivdb = requests.post('https://hivdb.stanford.edu/graphql', json=payload)
        try:
            subtype = hivdb.json().get('data').get('sequenceAnalysis')[0].get('bestMatchingSubtype').get('display')[0]
            print(f'{id} - subtype {subtype} (guessed)')
        except:
            print(hivdb.text)


In [27]:
fa = read_fasta('O:\\OneDrive\\Projects\\Project-HIV_analysis_pipeline\\workflow\\references\\HIV1_Genomes_32Strains.fasta')
print(f'Loaded: {len(fa.keys())} sequence(s)')
for id in fa.keys():
    print(f'{id} - {get_subtype(id, fa[id].seq)}')

Loaded: 32 sequence(s)
AF164485.1 - subtype E
AF259954.1 - subtype E
AF259955.1 - subtype E
KU168309.1 - CRF01_AE
U51189.1 - subtype E
AB049811.1 - subtype A
AB231895.1 - subtype G
KC492737 - CRF07_BC
KC492738 - CRF07_BC
KC503852 - CRF07_BC
KC503853 - CRF07_BC
KC503854 - CRF07_BC
KC503855 - CRF07_BC
KF234628 - CRF07_BC
AF385934 - CRF12_BF
AF385935 - CRF12_BF
AF385936 - CRF12_BF
AB703607.1 - CRF35_AD
EF158040.1 - CRF35_AD
AF324493.2 - NL4-3
MH705157.1 - subtype A
AY835759.1 - subtype B
K03455.1 - HXB2
X01762.1 - subtype B (99.22562478000704%)
AY772699.1 - subtype C
AB485648.1 - subtype D (94.61457233368532%)
AJ320484.1 - subtype 92UG001.
KY392769.1 - subtype D (100.0%)
AB231893.1 - subtype G
AB485662.1 - subtype G (100.0%)
AF084936.1 - subtype G
MH705134.1 - subtype G


In [31]:
for id in fa.keys():
    print(f'{id} - {serch_hivdb(fa[id].seq)}')

AF164485.1 - subtype CRF01_AE (98.38085181274198%)
AF259954.1 - subtype CRF01_AE (100.0%)
AF259955.1 - subtype CRF01_AE (100.0%)
KU168309.1 - subtype CRF01_AE (97.571277719113%)
U51189.1 - subtype CRF01_AE (98.80323829637452%)
AB049811.1 - subtype CRF02_AG (100.0%)
AB231895.1 - subtype CRF02_AG (99.82400563181977%)
KC492737 - subtype CRF07_BC (97.9936642027455%)
KC492738 - subtype CRF07_BC (97.95846532910947%)
KC503852 - subtype CRF07_BC (97.78247096092925%)
KC503853 - subtype CRF07_BC (97.7472720872932%)
KC503854 - subtype CRF07_BC (97.85286870820133%)
KC503855 - subtype CRF07_BC (97.8176698345653%)
KF234628 - subtype CRF07_BC (98.13445969728969%)
AF385934 - subtype CRF12_BF (100.0%)
AF385935 - subtype CRF12_BF (96.83210137275607%)
AF385936 - subtype CRF12_BF (96.58570925730376%)
AB703607.1 - subtype CRF35_AD (100.0%)
EF158040.1 - subtype CRF35_AD (97.92326645547342%)
AF324493.2 - subtype B (98.20485744456178%)
MH705157.1 - subtype A (95.91693065821893%)
AY835759.1 - subtype B (98.592